/Users/bolevard/ML/practice/nsfw_project/CLIP
Obtaining file:///Users/bolevard/ML/practice/nsfw_project/CLIP
  Preparing metadata (setup.py) ... done
  Attempting uninstall: clip
    Found existing installation: clip 1.0
    Uninstalling clip-1.0:
      Successfully uninstalled clip-1.0
  Running setup.py develop for clip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.8 -m pip install --upgrade pip
/Users/bolevard/ML/practice/nsfw_project


In [41]:
import pytesseract
import cv2
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoProcessor
from torch import nn
from torchvision import transforms, models
from PIL import Image
from torch.autograd import Variable
from itertools import islice
import warnings

% cd
CLIP
!pip
install - e.
% cd..

import CLIP.clip as clip


device = 'cpu'
config = '-l rus+eng'
path = ['sensored/bigboys.jpg']
example_text_from_post = 'Тестовый текст'


class Main:
    def __init__(self, path_list, text_from_post):
        self.emergency_flag = 0
        self.text_from_post = text_from_post
        self.path = './pytorch_nsfw_model-master/ResNet50_nsfw_model.pth'
        self.classes = ['drawings', 'hentai', 'neutral', 'porn', 'sexy']
        self.exceptions = ['drawings', 'neutral']
        self.text_list = ['Photo without violence and killing a person or animal',
                          'Photo with violence and killing of a person or animal or blood']

        self.path_list = path_list
        self.pil_images = [Image.open(x) for x in self.path_list]

        self.img = [cv2.imread(x) for x in self.path_list]
        self.img = [cv2.cvtColor(x, cv2.COLOR_BGR2RGB) for x in self.img]

        self.transformation = transforms.Compose([transforms.Resize(224),
                                                  transforms.CenterCrop(224),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                       std=[0.229, 0.224, 0.225])
                                                  ])

        # ResNet 50 model for NSFW detecting
        try:
            self.image_model = models.resnet50()
            self.image_model.fc = nn.Sequential(nn.Linear(2048, 512),
                                                nn.ReLU(),
                                                nn.Dropout(0.2),
                                                nn.Linear(512, 10),
                                                nn.LogSoftmax(dim=1))
            self.image_model.load_state_dict(torch.load(self.path, map_location=device))
            self.image_model.eval()

        except Exception:
            warnings.warn("An error occurred while loading the ResNet model")
            self.emergency_flag = 1

        # CLIP model for violence detecting
        try:
            self.clip = clip
            self.clip_model, self.clip_transform = clip.load("ViT-B/32", device=device)
        except RuntimeError:
            warnings.warn("An error occurred while loading the Clip model")
            self.emergency_flag = 1

        # detecting text in an image using PyTesseract model
        try:
            self.string = ' '.join(
                [pytesseract.image_to_string(x, config=config).replace('\n', ' ').strip() for x in self.img])
            self.texts = [self.string, self.text_from_post]
        except Exception:
            warnings.warn("An error occurred while using PyTesseract model")
            self.emergency_flag = 1

        # NLP model for detecting aggression in text
        try:
            self.nlp_tokenizer = AutoTokenizer.from_pretrained('nlp_tokenizer/')
            self.nlp_model = AutoModelForSequenceClassification.from_pretrained('nlp_model/').to(device)

            self.tokenized_text = self.nlp_tokenizer(
                self.texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors='pt'
            )
            self.tokenized_text = dict(map(lambda x: (x[0], (x[1]).to(device)), self.tokenized_text.items()))

        except Exception:
            warnings.warn("An error occurred while loading the NLP model")
            self.emergency_flag = 1

    def predict_image(self, image):
        image_tensor = self.transformation(image).float()
        image_tensor = image_tensor.unsqueeze_(0)

        if torch.cuda.is_available():
            image_tensor.cuda()

        input_ = Variable(image_tensor)
        output = self.image_model(input_)

        return torch.softmax(output[:, :5], 1)

    def get_logits(self, images_path_list, text):
        clip_images = [Image.open(x) for x in images_path_list]
        transformed_images = [self.clip_transform(x).unsqueeze(0).to(device) for x in clip_images]
        tokenized_text = self.clip.tokenize(text).to(device)
        with torch.no_grad():
            output = torch.cat([self.clip_model(image, tokenized_text)[0] for image in transformed_images], dim=0)
        probs = torch.softmax(output, dim=1)[:, 1].cpu().tolist()
        return probs

    def predict(self):
        self.image_dict = [self.predict_image(Image.open(x)) for x in self.path_list]
        self.image_dict = dict(zip(path, [dict(zip(self.classes, x[0].float().tolist())) for x in self.image_dict]))
        self.logits = self.nlp_model(
            **self.tokenized_text
        ).logits

        violence = self.get_logits(self.path_list, self.text_list)
        for name, proba in zip(self.path_list, violence):
            self.image_dict[name]['violence'] = proba

        self.image_dict['text'] = {}
        self.image_dict['text']['image_text_agressive'], \
        self.image_dict['text']['main_text_agressive'] = torch.softmax(self.logits, 1)[:, 1].tolist()
        return self.image_dict

    def moderate(self):
        if self.emergency_flag:
            return -1, {}
        flag = 0
        final_dict = self.predict()
        for name, dict_ in dict(islice(final_dict.items(), len(final_dict) - 1)).items():
            for type_, proba in dict_.items():
                if type_ not in self.exceptions:
                    if proba > 0.6:
                        flag = 1
        if final_dict['text']['image_text_agressive'] > 0.6 or final_dict['text']['main_text_agressive'] > 0.6:
            flag = 1
        return flag, final_dict


model = Main(path, example_text_from_post)
model.moderate()

/Users/bolevard/ML/practice/nsfw_project/CLIP
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Obtaining file:///Users/bolevard/ML/practice/nsfw_project/CLIP
  Preparing metadata (setup.py) ... done
  Attempting uninstall: clip
    Found existing installation: clip 1.0
    Uninstalling clip-1.0:
      Successfully uninstalled clip-1.0
  Running setup.py develop for clip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.8 -m pip install --upgrade pip
/Users/bolevard/ML/practice/nsfw_project
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either

(0,
 {'sensored/bigboys.jpg': {'drawings': 0.0012392222415655851,
   'hentai': 0.0011606772895902395,
   'neutral': 0.67901611328125,
   'porn': 0.27151164412498474,
   'sexy': 0.04707230255007744,
   'violence': 0.5673182606697083},
  'text': {'image_text_agressive': 0.014444471336901188,
   'main_text_agressive': 0.10063949227333069}})

1
